In [1]:
import pickle
file = open('train_feature.txt','rb')

timeseries_features_new = pickle.load(file)
timeseries_features_label_new = pickle.load(file)
timeseries_features_label_vital_new = pickle.load(file)
scaler_list_new = pickle.load(file)

In [2]:
print(timeseries_features_new.shape,timeseries_features_label_new.shape,timeseries_features_label_vital_new.shape)

(2450341, 128) (2450341,) (2450341,)


In [3]:
print(sum(timeseries_features_label_new),sum(timeseries_features_label_vital_new))

53219 4853


In [4]:
print(len(scaler_list_new),scaler_list_new)

26 [StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardScaler(copy=True, with_mean=True, with_std=True), StandardSc

In [5]:
file.close()

In [6]:
file = open('test_feature.txt','rb')
testseries_features_new = pickle.load(file)
file.close()

In [7]:
print(testseries_features_new.shape)

(2319237, 128)


In [8]:
import os
import numpy as np   
import sklearn
import keras

Using TensorFlow backend.


In [9]:
import pandas as pd

In [10]:
train_data = pd.read_csv('../../input/train.csv')
test_data = pd.read_csv('../../input/test.csv')

In [11]:
import random

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

np.random.seed(5)

In [13]:
file = open('train_test_all.txt','rb')
timeseries_all = pickle.load(file)
timeseries_label = pickle.load(file)
testseries_all = pickle.load(file)
file.close()

In [14]:
for i in range(len(timeseries_all)):
    print(len(timeseries_all[i]),len(testseries_all[i]))

128562 112627
128613 110185
129035 112113
128679 108747
129128 111171
128971 111004
128667 112786
128853 112116
8784 8784
10960 10960
8248 8249
8247 8248
8784 8784
8784 8784
8784 8784
147024 147024
147009 147010
100254 108901
147629 147629
147668 147669
147689 147690
137925 137925
129453 129454
147680 147681
65449 65450
65436 65436


In [15]:
import time

In [17]:
data_features_diff = len(test_data) - len(testseries_features_new)
print(data_features_diff)
data_features_diff_avg = int(data_features_diff / len(testseries_all))
print(data_features_diff_avg)

25974
999


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [22]:
help(StratifiedKFold)

Help on class StratifiedKFold in module sklearn.model_selection._split:

class StratifiedKFold(_BaseKFold)
 |  StratifiedKFold(n_splits='warn', shuffle=False, random_state=None)
 |  
 |  Stratified K-Folds cross-validator
 |  
 |  Provides train/test indices to split data in train/test sets.
 |  
 |  This cross-validation object is a variation of KFold that returns
 |  stratified folds. The folds are made by preserving the percentage of
 |  samples for each class.
 |  
 |  Read more in the :ref:`User Guide <cross_validation>`.
 |  
 |  Parameters
 |  ----------
 |  n_splits : int, default=3
 |      Number of folds. Must be at least 2.
 |  
 |      .. versionchanged:: 0.20
 |          ``n_splits`` default value will change from 3 to 5 in v0.22.
 |  
 |  shuffle : boolean, optional
 |      Whether to shuffle each class's samples before splitting into batches.
 |  
 |  random_state : int, RandomState instance or None, optional, default=None
 |      If int, random_state is the seed used by

In [96]:
ratio = round((len(timeseries_features_label_new) - sum(timeseries_features_label_new)) * 0.1 / sum(timeseries_features_label_new))
print(ratio)
non_anomaly = np.ones(len(timeseries_features_label_new)) - timeseries_features_label_new
print(non_anomaly,non_anomaly.shape)
sample_ratio = (99*ratio) * timeseries_features_label_vital_new + non_anomaly
print(sample_ratio,sum(sample_ratio))
sample_ratio = sample_ratio + ratio * timeseries_features_label_new
print(sample_ratio,sum(sample_ratio))

5.0
[1. 1. 1. ... 0. 0. 0.] (2450341,)
[1. 1. 1. ... 0. 0. 0.] 4799357.0
[1. 1. 1. ... 5. 5. 5.] 5065452.0


In [23]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=5)

In [34]:
count = 0
for train_index, test_index in kfold.split(timeseries_features_new, timeseries_features_label_new):
    count += 1
    m = Sequential()
    m.add(Dense(128, input_dim = 128, kernel_regularizer=l2(0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(Dropout(0.5))

    m.add(Dense(64, kernel_regularizer=l2(0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(Dropout(0.5))

    m.add(Dense(1, kernel_regularizer=l2(0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('sigmoid'))


    m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    
    h = m.fit(timeseries_features_new[train_index], timeseries_features_label_new[train_index], epochs=10, batch_size=5000, verbose=1,
                   sample_weight=sample_ratio[train_index])
    print("Round",count,":\n\n")
    p_train = m.predict(timeseries_features_new[train_index], batch_size=5000,verbose=1)
    p_test = m.predict(timeseries_features_new[test_index], batch_size=5000,verbose=1)
    t_list = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
    for threshhold in t_list:
        train_data_check = np.ravel(p_train > threshhold).astype(int)
        print("threshhold:",threshhold,"training data")
        print(precision_score(timeseries_features_label_new[train_index], train_data_check))
        print(recall_score(timeseries_features_label_new[train_index], train_data_check))
        print(f1_score(timeseries_features_label_new[train_index], train_data_check))
        test_data_check = np.ravel(p_test > threshhold).astype(int)
        print("test data")
        print(precision_score(timeseries_features_label_new[test_index], test_data_check))
        print(recall_score(timeseries_features_label_new[test_index], test_data_check))
        print(f1_score(timeseries_features_label_new[test_index], test_data_check))

Epoch 1/10
2205306/2205306 [==============================] - 25s 11us/step - loss: 0.9727 - binary_accuracy: 0.7564
Epoch 2/10
2205306/2205306 [==============================] - 19s 9us/step - loss: 0.7813 - binary_accuracy: 0.8538
Epoch 3/10
2205306/2205306 [==============================] - 19s 9us/step - loss: 0.6828 - binary_accuracy: 0.8896
Epoch 4/10
2205306/2205306 [==============================] - 19s 9us/step - loss: 0.6176 - binary_accuracy: 0.9060
Epoch 5/10
2205306/2205306 [==============================] - 19s 9us/step - loss: 0.5741 - binary_accuracy: 0.9157
Epoch 6/10
2205306/2205306 [==============================] - 19s 9us/step - loss: 0.5438 - binary_accuracy: 0.9243
Epoch 7/10
2205306/2205306 [==============================] - 19s 9us/step - loss: 0.5264 - binary_accuracy: 0.9285
Epoch 8/10
2205306/2205306 [==============================] - 19s 9us/step - loss: 0.5057 - binary_accuracy: 0.9329
Epoch 9/10
2205306/2205306 [==============================] - 19s 9us/s

KeyboardInterrupt: 

In [68]:
from keras.regularizers import l1

In [157]:
m = Sequential()
m.add(Dense(128, input_dim = 128))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(Dropout(0.5))

m.add(Dense(64))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(Dropout(0.5))

m.add(Dense(1))
m.add(BatchNormalization())
m.add(Activation('sigmoid'))


m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

h = m.fit(timeseries_features_new, timeseries_features_label_new, epochs=30, batch_size=5000, verbose=1,
               sample_weight=sample_ratio_new)

Epoch 1/30
2450341/2450341 [==============================] - 31s 13us/step - loss: 0.9051 - binary_accuracy: 0.7722
Epoch 2/30
2450341/2450341 [==============================] - 21s 9us/step - loss: 0.7713 - binary_accuracy: 0.8821
Epoch 3/30
2450341/2450341 [==============================] - 22s 9us/step - loss: 0.7127 - binary_accuracy: 0.9030
Epoch 4/30
2450341/2450341 [==============================] - 22s 9us/step - loss: 0.6773 - binary_accuracy: 0.9111
Epoch 5/30
2450341/2450341 [==============================] - 21s 9us/step - loss: 0.6538 - binary_accuracy: 0.9145
Epoch 6/30
2450341/2450341 [==============================] - 21s 9us/step - loss: 0.6326 - binary_accuracy: 0.9162
Epoch 7/30
2450341/2450341 [==============================] - 22s 9us/step - loss: 0.6103 - binary_accuracy: 0.9171
Epoch 8/30
2450341/2450341 [==============================] - 21s 9us/step - loss: 0.5952 - binary_accuracy: 0.9169
Epoch 9/30
2450341/2450341 [==============================] - 21s 9us/s

In [158]:
p = m.predict(timeseries_features_new, batch_size=5000,verbose=1)

2450341/2450341 [==============================] - 9s 4us/step


In [553]:
train_data_check = np.ravel(p>0.96).astype(int)
print(sum(train_data_check)/len(train_data_check))
print(precision_score(timeseries_features_label_new, train_data_check))
print(recall_score(timeseries_features_label_new, train_data_check))
print(f1_score(timeseries_features_label_new, train_data_check))

0.0031930249708101853
0.7627811860940695
0.11214040098461076
0.19553429549661713


In [554]:
last_index = 0
evaluation_new = np.zeros(data_features_diff_avg).astype(int)
next_index = 0
for i in range(len(timeseries_all)):
    next_index += len(timeseries_all[i]) - data_features_diff_avg
    evaluation_new = np.concatenate((evaluation_new, train_data_check[last_index : next_index]))
    print(len(evaluation_new),next_index)
    last_index = next_index
    if i != len(timeseries_all)-1:
        evaluation_new = np.concatenate((evaluation_new,np.zeros(data_features_diff_avg)))
print(len(evaluation_new))
assert(len(evaluation_new) == len(train_data))
evaluation_new = evaluation_new.astype(int)
evaluation_df = pd.DataFrame({'KPI ID': train_data['KPI ID'], 
                         'timestamp': train_data['timestamp'], 
                         'predict': evaluation_new})
evaluation_df.to_csv('evaluation.csv', index=False)

128562 127563
257175 255177
386210 383213
514889 510893
644017 639022
772988 766994
901655 894662
1030508 1022516
1039292 1030301
1050252 1040262
1058500 1047511
1066747 1054759
1075531 1062544
1084315 1070329
1093099 1078114
1240123 1224139
1387132 1370149
1487386 1469404
1635015 1616034
1782683 1762703
1930372 1909393
2068297 2046319
2197750 2174773
2345430 2321454
2410879 2385904
2476315 2450341
2476315


In [555]:
!python evaluation.py "../../input/train.csv" "evaluation.csv" 7

OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS proc 0 maps to package 0 
OMP: Info #250: KMP_AFFINITY: pid 47124 tid 47124 thread 0 bound to OS proc set 0
{"result": true, "data": 0.6806020664751941, "message": "计算成功"}


In [320]:
pm_t = m.predict(testseries_features_new,batch_size=5000,verbose=1)

2319237/2319237 [==============================] - 7s 3us/step


In [552]:
predict_flagm = (np.ravel(pm_t)>0.96).astype(int)
print(predict_flagm)
print(sum(predict_flagm)/len(predict_flagm))

[0 0 0 ... 0 0 0]
0.004897300275909707


In [178]:
last_index = 0
predict_new = np.zeros(data_features_diff_avg).astype(int)
next_index = 0
for i in range(len(testseries_all)):
    next_index += len(testseries_all[i]) - data_features_diff_avg
    predict_new = np.concatenate((predict_new, predict_flagm[last_index : next_index]))
    print(next_index)
    last_index = next_index
    if i != len(testseries_all)-1:
        predict_new = np.concatenate((predict_new,np.zeros(data_features_diff_avg)))
print(len(predict_new))
assert(len(predict_new) == len(test_data))
predict_new = predict_new.astype(int)
predict_df = pd.DataFrame({'KPI ID': test_data['KPI ID'], 
                         'timestamp': test_data['timestamp'], 
                         'predict': predict_new})
predict_df.to_csv('predicta5.csv', index=False)

111628
220814
331928
439676
549848
659853
771640
882757
890542
900503
907753
915002
922787
930572
938357
1084382
1230393
1338295
1484925
1631595
1778286
1915212
2043667
2190349
2254800
2319237
2345211


In [149]:
def get_first_delay_label(label, delay):
    splits = np.where(label[1:] != label[:-1])[0] + 1
    vital_label = np.zeros(len(label)).astype(int)
    is_anomaly = label[0] == 1
    pos = 0
    for sp in splits:
        if is_anomaly:
            vital_label[pos:min(pos + delay + 1, sp)] = 1
        #print(is_anomaly, pos, min(pos + delay + 1, sp),
         #     vital_label[pos:min(pos + delay + 1, sp)],label[pos:min(pos + delay + 1, sp)])
        is_anomaly = not is_anomaly
        pos = sp
    sp = len(label)
    if is_anomaly:
        vital_label[pos:min(pos + delay + 1, sp)] = 1
    return vital_label
        

In [151]:
vital_label = get_first_delay_label(timeseries_features_label_new,7)
print(sum(vital_label),len(vital_label))

5189 2450341


In [141]:
print(sum(timeseries_features_label_vital_new))

4853


In [153]:
sum_non = len(timeseries_features_label_new) - sum(timeseries_features_label_new)
vital_ratio = round((sum_non - sum(timeseries_features_label_new) + sum(vital_label))/sum(vital_label))
print(sum_non,sum(timeseries_features_label_new),sum(vital_label),vital_ratio)

2397122 53219 5189 453.0


In [381]:
sample_ratio_new = vital_ratio * vital_label + 1
print(sum(sample_ratio_new))
print(sample_ratio_new)

4800958.0
[1. 1. 1. ... 1. 1. 1.]


In [180]:
import xgboost as xgb

In [281]:
count = 0
for train_index, test_index in kfold.split(timeseries_features_new, timeseries_features_label_new):
    count += 1
    xgb_model = xgb.XGBClassifier(n_jobs=10, verbosity=1)
    
    xgb_model.fit(timeseries_features_new, timeseries_features_label_new, 
                  sample_weight = xgb_sample_ratio, verbose = True)
    print("Round",count,":\n\n")
    p_train = (xgb_model.predict_proba(timeseries_features_new[train_index]))[:,1:]
    p_test = (xgb_model.predict_proba(timeseries_features_new[test_index]))[:,1:]
    t_list = [0.7, 0.8, 0.9, 0.95, 0.99]
    for threshhold in t_list:
        train_data_check = np.ravel(p_train > threshhold).astype(int)
        print("threshhold:",threshhold)
        print("training data")
        print(precision_score(timeseries_features_label_new[train_index], train_data_check))
        print(recall_score(timeseries_features_label_new[train_index], train_data_check))
        print(f1_score(timeseries_features_label_new[train_index], train_data_check))
        
        test_data_check = np.ravel(p_test > threshhold).astype(int)
        print("test data")
        print(precision_score(timeseries_features_label_new[test_index], test_data_check))
        print(recall_score(timeseries_features_label_new[test_index], test_data_check))
        print(f1_score(timeseries_features_label_new[test_index], test_data_check))
    break

Round 1 :


threshhold: 0.7
training data
0.5645237697152092
0.8459193686452179
0.6771511418997401
test data
0.5681619392727727
0.8543780533633972
0.6824765478424015
threshhold: 0.8
training data
0.716100659662839
0.7955195523727999
0.7537238146104088
test data
0.720536462699078
0.8075911311537016
0.7615841233277222
threshhold: 0.9
training data
0.8663378411339061
0.6674113201244337
0.7539742440681165
test data
0.867275787448906
0.6777527245396467
0.7608902014555426
threshhold: 0.95
training data
0.9271312977781752
0.4329916278681337
0.5902997182136452
test data
0.9203264671589584
0.444945509207065
0.5998733375554148
threshhold: 0.99
training data
0.9914738124238733
0.11896360941186296
0.21243755126388789
test data
0.9940209267563528
0.12495302517850432
0.22199966616591552


In [282]:
xgb_model = xgb.XGBClassifier(n_jobs=10, verbosity=2)

In [283]:
ratio = round((len(timeseries_features_label_new) - sum(timeseries_features_label_new)) * 0.5 / sum(timeseries_features_label_new))
print(ratio)
non_anomaly = np.ones(len(timeseries_features_label_new)) - timeseries_features_label_new
print(non_anomaly,non_anomaly.shape)
xgb_sample_ratio = (9*ratio) * vital_label + non_anomaly
print(xgb_sample_ratio,sum(xgb_sample_ratio))
xgb_sample_ratio = xgb_sample_ratio + ratio * timeseries_features_label_new
print(xgb_sample_ratio,sum(xgb_sample_ratio))

23.0
[1. 1. 1. ... 0. 0. 0.] (2450341,)
[1. 1. 1. ... 0. 0. 0.] 3471245.0
[ 1.  1.  1. ... 23. 23. 23.] 4695282.0


In [284]:
xgb_model.fit(timeseries_features_new, timeseries_features_label_new, sample_weight = xgb_sample_ratio, verbose = True)

[10:40:19] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:40:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:40:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:40:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:40:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:40:34] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:40:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:40:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nod

[10:43:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:43:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:43:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:43:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:43:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:43:34] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:43:36] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:43:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nod

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=10,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=2)

In [285]:
y_p = xgb_model.predict_proba(timeseries_features_new)

In [399]:
train_data_check = (np.ravel(y_p[:,1:])>0.98).astype(int)
print(train_data_check.shape)
print(sum(train_data_check)/len(train_data_check))
print(precision_score(timeseries_features_label_new, train_data_check))
print(recall_score(timeseries_features_label_new, train_data_check))
print(f1_score(timeseries_features_label_new, train_data_check))

(2450341,)
0.006153021150933686
0.9822245804868343
0.27826528119656513
0.43367107883331374


In [557]:
last_index = 0
evaluation_new = np.zeros(data_features_diff_avg).astype(int)
next_index = 0
for i in range(len(timeseries_all)):
    next_index += len(timeseries_all[i]) - data_features_diff_avg
    evaluation_new = np.concatenate((evaluation_new, train_data_check[last_index : next_index]))
    print(len(evaluation_new),next_index)
    last_index = next_index
    if i != len(timeseries_all)-1:
        evaluation_new = np.concatenate((evaluation_new,np.zeros(data_features_diff_avg)))
print(len(evaluation_new))
assert(len(evaluation_new) == len(train_data))
evaluation_new = evaluation_new.astype(int)
evaluation_df = pd.DataFrame({'KPI ID': train_data['KPI ID'], 
                         'timestamp': train_data['timestamp'], 
                         'predict': evaluation_new})
evaluation_df.to_csv('evaluation.csv', index=False)

128562 127563
257175 255177
386210 383213
514889 510893
644017 639022
772988 766994
901655 894662
1030508 1022516
1039292 1030301
1050252 1040262
1058500 1047511
1066747 1054759
1075531 1062544
1084315 1070329
1093099 1078114
1240123 1224139
1387132 1370149
1487386 1469404
1635015 1616034
1782683 1762703
1930372 1909393
2068297 2046319
2197750 2174773
2345430 2321454
2410879 2385904
2476315 2450341
2476315


In [558]:
!python evaluation.py "../../input/train.csv" "evaluation.csv" 7

OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS proc 0 maps to package 0 
OMP: Info #250: KMP_AFFINITY: pid 17116 tid 17116 thread 0 bound to OS proc set 0
{"result": true, "data": 0.8314651303567328, "message": "计算成功"}


In [314]:
p_t = xgb_model.predict_proba(testseries_features_new)

In [504]:
predict_flag = (np.ravel(p_t[:,1:])>0.98).astype(int)
print(predict_flag)
print(sum(predict_flag)/len(predict_flag))

[0 0 0 ... 0 0 0]
0.0023279207773936


In [550]:
predict_t1 = (np.ravel(p_t[:,1:])>0.98)
predict_t2 = np.ravel(pm_t)>0.96
predict_t3 = (np.ravel(p_test_test[:,1:]) > 0.9325)
predict_xg = ((predict_t1 | predict_t3)).astype(int)
predict_flag = (predict_t2 | predict_xg).astype(int)
print(sum(predict_t2)/len(predict_t2), sum(predict_xg)/len(predict_xg))
print(predict_flag)
print(sum(predict_flag)/len(predict_flag))

0.004897300275909707 0.004651529791910012
[0 0 0 ... 0 0 0]
0.0072454863388260885


In [551]:
last_index = 0
predict_new = np.zeros(data_features_diff_avg).astype(int)
next_index = 0
for i in range(len(testseries_all)):
    next_index += len(testseries_all[i]) - data_features_diff_avg
    predict_new = np.concatenate((predict_new, predict_flag[last_index : next_index]))
    print(next_index)
    last_index = next_index
    if i != len(testseries_all)-1:
        predict_new = np.concatenate((predict_new,np.zeros(data_features_diff_avg)))
print(len(predict_new))
assert(len(predict_new) == len(test_data))
predict_new = predict_new.astype(int)
predict_df = pd.DataFrame({'KPI ID': test_data['KPI ID'], 
                         'timestamp': test_data['timestamp'], 
                         'predict': predict_new})
predict_df.to_csv('predict.csv', index=False)

111628
220814
331928
439676
549848
659853
771640
882757
890542
900503
907753
915002
922787
930572
938357
1084382
1230393
1338295
1484925
1631595
1778286
1915212
2043667
2190349
2254800
2319237
2345211


In [254]:
train_temp1 = np.ravel(p)
print(train_temp1.shape)
train_temp2 = np.ravel(y_p[:,1:])
print(train_temp2.shape)

(2450341,)
(2450341,)


In [255]:
train_data_check = ((train_temp1*0.7 + train_temp2 * 0.3)>0.9).astype(int)
print(train_data_check)
print(sum(train_data_check)/len(train_data_check))
print(precision_score(timeseries_features_label_new, train_data_check))
print(recall_score(timeseries_features_label_new, train_data_check))
print(f1_score(timeseries_features_label_new, train_data_check))

[0 0 0 ... 0 0 0]
0.007216954701406865
0.8724836009952499
0.2899152558296849
0.43521430686994916


In [204]:
predict_flagm = (np.ravel(pm_t)>0.85).astype(int)
print(predict_flagm)
print(sum(predict_flagm)/len(predict_flagm))

[0 0 0 ... 0 0 0]
0.011411942807052492


In [209]:
temp1 = np.ravel(pm_t)
print(temp1)

[0.221663   0.13683766 0.11092606 ... 0.16910532 0.16221571 0.15728143]


In [258]:
predict_flag = (np.ravel(p_t[:,1:])>0.9).astype(int)
print(predict_flag)
print(sum(predict_flag)/len(predict_flag))

[0 0 0 ... 0 0 0]
0.011036819436737169


In [259]:
temp2 = np.ravel(p_t[:,1:])
print(temp2)

[0.11381663 0.21982214 0.19585699 ... 0.6884514  0.71765184 0.6952185 ]


In [262]:
predict_flag = ((temp1*0.7 + temp2 * 0.3)>0.85).astype(int)
print(predict_flag)
print(sum(predict_flag)/len(predict_flag))

[0 0 0 ... 0 0 0]
0.00589633573455408


In [263]:
last_index = 0
predict_new = np.zeros(data_features_diff_avg).astype(int)
next_index = 0
for i in range(len(testseries_all)):
    next_index += len(testseries_all[i]) - data_features_diff_avg
    predict_new = np.concatenate((predict_new, predict_flag[last_index : next_index]))
    print(next_index)
    last_index = next_index
    if i != len(testseries_all)-1:
        predict_new = np.concatenate((predict_new,np.zeros(data_features_diff_avg)))
print(len(predict_new))
assert(len(predict_new) == len(test_data))
predict_new = predict_new.astype(int)
predict_df = pd.DataFrame({'KPI ID': test_data['KPI ID'], 
                         'timestamp': test_data['timestamp'], 
                         'predict': predict_new})
predict_df.to_csv('predictb3.csv', index=False)

111628
220814
331928
439676
549848
659853
771640
882757
890542
900503
907753
915002
922787
930572
938357
1084382
1230393
1338295
1484925
1631595
1778286
1915212
2043667
2190349
2254800
2319237
2345211


In [242]:
file = open('xgboost.txt','rb')
xgb_model_original = pickle.load(file)
file.close()

In [380]:
ratio = round((len(timeseries_features_label_new) - sum(timeseries_features_label_new)) * 0.05 / sum(timeseries_features_label_new))
print(ratio)
non_anomaly = np.ones(len(timeseries_features_label_new)) - timeseries_features_label_new
print(non_anomaly,non_anomaly.shape)
xgb_sample_ratio = (239*ratio) * vital_label + non_anomaly
print(xgb_sample_ratio,sum(xgb_sample_ratio))
xgb_sample_ratio = xgb_sample_ratio + ratio * timeseries_features_label_new
print(xgb_sample_ratio,sum(xgb_sample_ratio))

2.0
[1. 1. 1. ... 0. 0. 0.] (2450341,)
[1. 1. 1. ... 0. 0. 0.] 4981244.0
[1. 1. 1. ... 2. 2. 2.] 5087682.0


In [359]:
importance = xgb_model.feature_importances_
print(importance)

[5.9824758e-03 1.3047622e-03 0.0000000e+00 2.4752015e-02 0.0000000e+00
 1.6563017e-02 3.4112965e-03 1.2314280e-02 1.1824915e-02 1.1404434e-02
 3.4108406e-03 2.8897815e-03 0.0000000e+00 5.8032260e-03 0.0000000e+00
 1.1023236e-03 0.0000000e+00 5.0067063e-03 8.6255493e-03 4.7814506e-03
 1.9957749e-02 4.4241104e-02 2.2151593e-05 2.5610323e-03 2.1993625e-03
 1.7483914e-03 3.9343075e-03 4.6621081e-03 1.1593622e-02 1.4758931e-02
 5.0650006e-03 4.4379572e-06 1.3486209e-02 1.1500513e-01 0.0000000e+00
 5.8656516e-03 0.0000000e+00 8.6713806e-03 2.1227219e-03 1.4631709e-02
 8.9146066e-03 4.0580314e-02 8.6860061e-03 1.7616397e-02 1.0436235e-02
 3.2610077e-02 0.0000000e+00 6.3901916e-03 4.9417901e-07 2.1208527e-02
 0.0000000e+00 5.1374873e-03 1.6504455e-02 4.0854238e-02 0.0000000e+00
 4.4543757e-03 3.7033465e-03 2.6178181e-02 0.0000000e+00 3.2410277e-03
 2.8820281e-04 0.0000000e+00 2.1728808e-03 1.2191213e-03 7.3412429e-03
 2.8603785e-03 1.8407607e-03 4.6718251e-03 8.8167088e-03 8.7471111e-03
 4.513

In [559]:
sum1 = 0
sort_index = np.argsort(importance)
importance_index = []
for i in range(len(sort_index)):
    if importance[sort_index[len(importance)-1-i]] > 0.001:
        importance_index.append(sort_index[len(importance)-1-i])
        sum1 += importance[sort_index[len(importance)-1-i]]
        print(sort_index[len(importance)-1-i],importance[sort_index[len(importance)-1-i]])


33 0.11500513
116 0.0847902
70 0.045133222
21 0.044241104
53 0.040854238
41 0.040580314
45 0.032610077
57 0.026178181
3 0.024752015
49 0.021208527
20 0.01995775
113 0.018494658
43 0.017616397
5 0.016563017
52 0.016504455
29 0.014758931
90 0.014666984
39 0.014631709
32 0.013486209
7 0.01231428
92 0.011981871
8 0.011824915
28 0.011593622
9 0.011404434
44 0.010436235
40 0.008914607
68 0.008816709
69 0.008747111
42 0.008686006
37 0.008671381
18 0.008625549
75 0.008523343
81 0.008495416
105 0.008101095
89 0.007931908
101 0.0076546837
111 0.007592641
64 0.007341243
82 0.0067483154
47 0.0063901916
115 0.0061481404
0 0.005982476
77 0.0059000035
35 0.0058656516
13 0.005803226
73 0.0057535926
51 0.0051374873
30 0.0050650006
76 0.0050443662
17 0.0050067063
121 0.004859619
19 0.0047814506
67 0.004671825
27 0.004662108
125 0.0046361485
99 0.0045426986
55 0.0044543757
80 0.004110789
123 0.0039934297
112 0.0039425585
93 0.00393659
26 0.0039343075
56 0.0037033465
110 0.003686027
83 0.0035288983
6 0.00

In [383]:
xgb_model_test = xgb.XGBClassifier(n_jobs=10, verbosity=2, max_depth=5)

In [384]:
xgb_model_test.fit(timeseries_features_new[:,importance_index], timeseries_features_label_new, sample_weight = sample_ratio_new, verbose = True)

[10:56:36] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:56:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:56:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:56:50] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:56:54] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:56:58] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:57:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:57:07] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nod

[11:01:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[11:01:17] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[11:01:21] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[11:01:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[11:01:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[11:01:34] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[11:01:38] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[11:01:43] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nod

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=10,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=2)

In [385]:
y_test = xgb_model_test.predict_proba(timeseries_features_new[:,importance_index])

In [556]:
train_data_check = (np.ravel(y_test[:,1:])>0.93).astype(int)
print(train_data_check.shape, train_data_check)
print(sum(train_data_check)/len(train_data_check))
print(precision_score(timeseries_features_label_new, train_data_check))
print(recall_score(timeseries_features_label_new, train_data_check))
print(f1_score(timeseries_features_label_new, train_data_check))

(2450341,) [0 0 0 ... 0 0 0]
0.00728429226789251
0.8610006162810242
0.28876904864803926
0.4324871953621884


In [382]:
help(xgb.XGBClassifier)

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, silent=None, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  Parameters
 |  ----------
 |  max_depth : int
 |      Maximum tree depth for base learners.
 |  learning_rate : float
 |      Boosting learning rate (xgb's "eta")
 |  n_estimators : int
 |      Number of trees to fit.
 |  verbosity : int
 |      The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 |  silent : boolean
 |      Whether to print messages while running boosting. Deprecat

In [393]:
p_test_test = xgb_model_test.predict_proba(testseries_features_new[:,importance_index])

In [507]:
predict_flag = (np.ravel(p_test_test[:,1:])>0.98).astype(int)
print(predict_flag)
print(sum(predict_flag)/len(predict_flag))

[0 0 0 ... 0 0 0]
0.002082581469681624


In [398]:
last_index = 0
predict_new = np.zeros(data_features_diff_avg).astype(int)
next_index = 0
for i in range(len(testseries_all)):
    next_index += len(testseries_all[i]) - data_features_diff_avg
    predict_new = np.concatenate((predict_new, predict_flag[last_index : next_index]))
    print(next_index)
    last_index = next_index
    if i != len(testseries_all)-1:
        predict_new = np.concatenate((predict_new,np.zeros(data_features_diff_avg)))
print(len(predict_new))
assert(len(predict_new) == len(test_data))
predict_new = predict_new.astype(int)
predict_df = pd.DataFrame({'KPI ID': test_data['KPI ID'], 
                         'timestamp': test_data['timestamp'], 
                         'predict': predict_new})
predict_df.to_csv('predict2.csv', index=False)

111628
220814
331928
439676
549848
659853
771640
882757
890542
900503
907753
915002
922787
930572
938357
1084382
1230393
1338295
1484925
1631595
1778286
1915212
2043667
2190349
2254800
2319237
2345211


In [456]:
xgb_model_new = xgb.XGBClassifier(n_jobs=10, verbosity=2, max_depth=5)

In [460]:
ratio = round((len(timeseries_features_label_new) - sum(vital_label)) / sum(vital_label))
print(ratio)
non_anomaly = np.ones(len(timeseries_features_label_new)) - timeseries_features_label_new
print(non_anomaly,non_anomaly.shape, sum(non_anomaly))
extremely_sample_ratio = non_anomaly + vital_label * ratio
print(extremely_sample_ratio,extremely_sample_ratio.shape, sum(extremely_sample_ratio))

471.0
[1. 1. 1. ... 0. 0. 0.] (2450341,) 2397122.0
[1. 1. 1. ... 0. 0. 0.] (2450341,) 4841141.0


In [467]:
print(sum(vital_label)/len(vital_label))

0.002117664439357624


In [461]:
xgb_model_new.fit(timeseries_features_new[:,importance_index], timeseries_features_label_new, sample_weight = extremely_sample_ratio, verbose = True)

[10:32:01] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:32:06] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:32:10] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[10:32:14] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:32:18] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:32:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:32:27] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:32:32] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nod

[10:36:36] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[10:36:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[10:36:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[10:36:49] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[10:36:54] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[10:36:58] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[10:37:02] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[10:37:07] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nod

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=10,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=2)

In [462]:
y_new = xgb_model_new.predict_proba(timeseries_features_new[:,importance_index])

In [471]:
train_data_check = (np.ravel(y_new[:,1:])>0.93).astype(int)
print(train_data_check.shape, train_data_check)
print(sum(train_data_check)/len(train_data_check))
print(precision_score(timeseries_features_label_new, train_data_check))
print(recall_score(timeseries_features_label_new, train_data_check))
print(f1_score(timeseries_features_label_new, train_data_check))

(2450341,) [0 0 0 ... 0 0 0]
0.003058757944302446
0.7934623082054704
0.111745805069618
0.19590209836281583


In [474]:
p_new = xgb_model_new.predict_proba(testseries_features_new[:,importance_index])

In [509]:
predict_flag = (np.ravel(p_new[:,1:])>0.98).astype(int)
print(predict_flag)
print(sum(predict_flag)/len(predict_flag))

[0 0 0 ... 0 0 0]
0.001924339772088838


In [498]:
last_index = 0
predict_new = np.zeros(data_features_diff_avg).astype(int)
next_index = 0
for i in range(len(testseries_all)):
    next_index += len(testseries_all[i]) - data_features_diff_avg
    predict_new = np.concatenate((predict_new, predict_flag[last_index : next_index]))
    print(next_index)
    last_index = next_index
    if i != len(testseries_all)-1:
        predict_new = np.concatenate((predict_new,np.zeros(data_features_diff_avg)))
print(len(predict_new))
assert(len(predict_new) == len(test_data))
predict_new = predict_new.astype(int)
predict_df = pd.DataFrame({'KPI ID': test_data['KPI ID'], 
                         'timestamp': test_data['timestamp'], 
                         'predict': predict_new})
predict_df.to_csv('predictnew.csv', index=False)

111628
220814
331928
439676
549848
659853
771640
882757
890542
900503
907753
915002
922787
930572
938357
1084382
1230393
1338295
1484925
1631595
1778286
1915212
2043667
2190349
2254800
2319237
2345211


In [483]:
model = Sequential()
model.add(Dense(128, input_dim = 128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

h = model.fit(timeseries_features_new, timeseries_features_label_new, epochs=30, batch_size=5000, verbose=1,
               sample_weight=extremely_sample_ratio)

Epoch 1/30
2450341/2450341 [==============================] - 34s 14us/step - loss: 1.0320 - binary_accuracy: 0.7401
Epoch 2/30
2450341/2450341 [==============================] - 24s 10us/step - loss: 0.8045 - binary_accuracy: 0.8787
Epoch 3/30
2450341/2450341 [==============================] - 23s 9us/step - loss: 0.7462 - binary_accuracy: 0.9032
Epoch 4/30
2450341/2450341 [==============================] - 24s 10us/step - loss: 0.7042 - binary_accuracy: 0.9097
Epoch 5/30
2450341/2450341 [==============================] - 23s 10us/step - loss: 0.6787 - binary_accuracy: 0.9093
Epoch 6/30
2450341/2450341 [==============================] - 23s 10us/step - loss: 0.6591 - binary_accuracy: 0.9131
Epoch 7/30
2450341/2450341 [==============================] - 24s 10us/step - loss: 0.6362 - binary_accuracy: 0.9149
Epoch 8/30
2450341/2450341 [==============================] - 24s 10us/step - loss: 0.6148 - binary_accuracy: 0.9125
Epoch 9/30
2450341/2450341 [==============================] - 23s

In [484]:
p_model = model.predict(timeseries_features_new, batch_size=5000,verbose=1)

2450341/2450341 [==============================] - 10s 4us/step


In [493]:
train_data_check = np.ravel(p_model>0.85).astype(int)
print(sum(train_data_check)/len(train_data_check))
print(precision_score(timeseries_features_label_new, train_data_check))
print(recall_score(timeseries_features_label_new, train_data_check))
print(f1_score(timeseries_features_label_new, train_data_check))

0.007050039157815178
0.4663386396526773
0.15137450910389147
0.2285584588759327


In [496]:
p_model_test = m.predict(testseries_features_new,batch_size=5000,verbose=1)

2319237/2319237 [==============================] - 7s 3us/step


In [502]:
predict_flag = (np.ravel(p_model_test)>0.99).astype(int)
print(predict_flag)
print(sum(predict_flag)/len(predict_flag))

[0 0 0 ... 0 0 0]
0.0037978007422268616
